# My First RAG Application in a Jupyter Notebook

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.11.4.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Here is the visual representation of the RAG application.

![Sample Image](images/rag_3.jpg)


## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [34]:
!pip install -qU numpy matplotlib plotly pandas scipy scikit-learn openai python-dotenv

In [35]:
!pip install PyMuPDF

Code to help autoreload changes to utilities

In [36]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio  # supports asynchronous tasks

In [38]:
import nest_asyncio  # allow asyncio loop to be reentered - important for Jupyter notebook
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:



### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.


In [39]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
print(f"Number of documents: {len(documents)}")
print(documents[0][:200])

Number of documents: 1
﻿
The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horowitz
cover design: Jessica Hagy
produced using: Pressbooks
Contents
THE PMARCA GUIDE TO STARTUPS
Part


In [40]:
# test .pdf document
text_loader = TextFileLoader("data/OOUX_launch_guide.pdf")
documents_pdf = text_loader.load_documents()
print(f"Number of documents: {len(documents_pdf)}")
print(documents_pdf[0][:200])

Number of documents: 1
Sophia Prater
Sophia Prater
A momentum-building orientation to the complexity-wrangling 
magic of Object-Oriented UX and the ORCA Process
The OOUX
LAUNCH GUIDE
Welcome to the wonderful world of OOUX!



In [41]:
import importlib
from gower import documentmanager
from aimakerspace import text_utils, vectordatabase
importlib.reload(documentmanager)
importlib.reload(text_utils)
importlib.reload(vectordatabase)

<module 'aimakerspace.vectordatabase' from '/home/rchrdgwr/Projects/code/AIE4/Week 1/Day 2/aimakerspace/vectordatabase.py'>

In [42]:
# text file directory
text_loader = TextFileLoader("./data/")
documents_folder = text_loader.load_documents()
print(f"Number of documents: {len(documents_folder)}")
for document in documents_folder:
    print(document[:50])
    print("---")

Number of documents: 5

The Saga of Erik the Red
1880 translation into En
---
﻿
The Pmarca Blog Archives
(select posts from 2007
---

Egil's Saga
1893 translation into English by W. C
---
Sophia Prater
Sophia Prater
A momentum-building or
---

The Story of Thórðr Hreða
1882 translation into E
---


### Splitting Text Into Chunks

As we can see, there is one massive document .

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [43]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)  # default, if not passed is 1000, 200
# text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=20)  # use to change the chunk size and overlap
split_documents = text_splitter.split_texts(documents)
len(split_documents)

373

Let's take a look at some of the documents we've managed to split.

In [44]:
print(split_documents[0:1])
print(split_documents[1:2])
print(split_documents[2:3])

['\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7: Why a startup\'s initial business plan doesn\'t\nmatter that much\n49\nTHE PMARCA GUIDE TO HIRING\nPart 8: Hiring, managing, promoting, and Dring\nexecutives\n54\nPart 9: How to hire a professional CEO 68\nHow to hire the best people you\'ve ever worked\nwith\n69\nTHE PMARCA GUIDE TO BIG COMPANIES\nPart 1: Turnaround! 82\nPart 2: Retaining great people 86\nTHE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\nAND SOME OTHER THINGS\nIntroduction 97\nPart 1: Opportunity 99\nPart 2: Skills and education 107\nPart 3: Where to go and wh

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [45]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

<div style="background-color: darkgreen; padding: 10px;">
<h1>❓Question #1:</h1>

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
    
    1. Answer: The best way is to use a model with a different model (simpler for smaller, more complex for higher). It is possible to 
    use the dimensions API parameter to shorten the embedding dimension.
    
2. What technique does OpenAI use to achieve this?
    
    2. Answer: Can shorten embeddings dimension using the dimensions API parameter. This effectively removes some numbers from the end of the sequence. The 
    newer models were trained using the Matryoshka Representation Learning that allows developers to shorten embeddings.


<br>
    
</div>
    
    

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #2!

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [46]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))
print(len(vector_db.vectors))
if vector_db.vectors:
    first_key, first_vector = next(iter(vector_db.vectors.items()))
    print(f"First key: {first_key}")
    print(f"First vector: {first_vector}")
else:
    print("The vector database is empty.")

373
First key: ﻿
The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horowitz
cover design: Jessica Hagy
produced using: Pressbooks
Contents
THE PMARCA GUIDE TO STARTUPS
Part 1: Why not to do a startup 2
Part 2: When the VCs say "no" 10
Part 3: "But I don't know any VCs!" 18
Part 4: The only thing that matters 25
Part 5: The Moby Dick theory of big companies 33
Part 6: How much funding is too little? Too much? 41
Part 7: Why a startup's initial business plan doesn't
matter that much
49
THE PMARCA GUIDE TO HIRING
Part 8: Hiring, managing, promoting, and Dring
executives
54
Part 9: How to hire a professional CEO 68
How to hire the best people you've ever worked
with
69
THE PMARCA GUIDE TO BIG COMPANIES
Part 1: Turnaround! 82
Part 2: Retaining great people 86
THE PMARCA GUIDE TO CAREER, PRODUCTIVITY,
AND SOME OTHER THINGS
Introduction 97
Part 1: Opportunity 99
Part 2: Skills and education 107
Part 3: Where to go and why 120
The Pmarca Guide to Pers

<div style="background-color: darkgreen; padding: 10px;">
<h1>❓Question #2:</h1>

What are the benefits of using an `async` approach to collecting our embeddings?

Answer 2 - the key benefit of using `async` approach is the is that it allows API calls to be made without blocking the main thread and supporting multiple calls at the same time.
This helps improve performance, scalability, as well as responsiveness for the user.

</div>

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

I have added support for Euclidian Distance and Minkowski Distance 

I was also going to implement annoy but was taking errors importing the annoy library so decided to bypass that for the moment

In [47]:
import numpy as np
def euclidean_distance(vector_a: np.array, vector_b: np.array) -> float:
    """Computes the Euclidean distance between two vectors."""
    distance = np.linalg.norm(vector_a - vector_b)
    return distance

def minkowski_distance(vector_a: np.array, vector_b: np.array, p: float = 3) -> float:
    """Computes the Minkowski distance between two vectors for a given order p.
        Default p=3 - a distance between Manhattan and Euclidean
        Note    1=Manhattan distance
                2=Euclidean distance
    """
    distance = np.sum(np.abs(vector_a - vector_b) ** p) ** (1 / p)
    return distance

def print_array(in_array):
    for item in in_array:
        print(item)

print("Default - Cosine Similarity")
print_array(vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", 
                         k=5))
print(" ")
print("Euclidean Distance")
print_array(vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", 
                         k=5, distance_measure=euclidean_distance))
print(" ")
print("Minkowski Distance")
print_array(vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", 
                         k=5, distance_measure=minkowski_distance))

Default - Cosine Similarity
('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

<div style="background-color: darkgreen; padding: 10px;">
<h1>❓ Question #3:</h1>

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

Answer 3: Setting a lower temperature score 0-2 - will result in more reproducible responses than a higher temperature score > 2 - note temperature is a range of 0-10.

</div>
> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

### Creating and Prompting OpenAI's `gpt-4o-mini`!

Let's tie all these together and use it to prompt `gpt-4o-mini`!

In [48]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
print(user_role_prompt)
print(    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ))
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)
print(system_role_prompt)
print(system_role_prompt.create_message(expertise="Javascript"))

messages = [
    system_role_prompt.create_message(expertise="linguistics"),
    user_role_prompt.create_message(
        content="What is the best way to learn icelandic?"
    ),
]

response = chat_openai.run(messages)

{'role': 'user', 'content': 'What is the best way to write a loop?'}
{'role': 'system', 'content': 'You are an expert in Javascript, you always answer in a kind way.'}


In [49]:
print(response)

Learning Icelandic can be a rewarding endeavor, given its unique linguistic features and rich cultural heritage! Here are some effective strategies to help you on your learning journey:

1. **Structured Courses**: Enroll in a language course, either online or in-person, that is specifically focused on Icelandic. Look for courses that emphasize speaking, listening, reading, and writing.

2. **Language Apps**: Use language learning apps such as Duolingo, Drops, or Memrise. These can provide a fun and interactive way to build your vocabulary and grammar skills.

3. **Textbooks and Grammar Guides**: Invest in a good textbook or grammar guide dedicated to Icelandic. This will help you understand the rules and structure of the language.

4. **Listening Practice**: Immerse yourself in the language by listening to Icelandic music, podcasts, and radio stations. This will help you improve your listening skills and become familiar with pronunciation.

5. **Reading**: Start with children’s books o

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [50]:
RAG_PROMPT_TEMPLATE = """ \
Use the provided context to answer the user's query.

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".
"""

rag_prompt = SystemRolePrompt(RAG_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

User Query:
{user_query}
"""


user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)
        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)

        return {"response" : self.llm.run([formatted_system_prompt, formatted_user_prompt]), "context" : context_list}

<div style="background-color: darkgreen; padding: 10px;">
<h1>❓ Question #4:</h1>

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

Answer 4: We could use "Chain of thought" approach to prompting by guiding the model to go through intermediate steps to arrive at an answer. We could provide sample answers to direct it to respond in a certain way.

</div>

> NOTE: You can look through the Week 1 Day 1 "Prompting OpenAI Like A Developer" material for an answer to this question!

Create the pipeline

In [51]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

In [54]:
result = retrieval_augmented_qa_pipeline.run_pipeline("What is the 'Michael Eisner Memorial Weak Executive Problem'?")
response = result["response"]
context_list = result["context"]
print("Response:")
print(response)
print("\nContext Used:")

# Print each context with its similarity score
for i, (context, score) in enumerate(context_list, 1):
    print(f"\nContext {i}:")
    print(f"Score: {score:.4f}")
    print(f"Text: {context.strip()}")


Response:
The 'Michael Eisner Memorial Weak Executive Problem' refers to the tendency of a CEO or a startup founder to hire a weak executive in their former area of expertise in order to maintain control and prominence in that function. This issue arises when the CEO, who has a strong background in a particular field (like product management, sales, or marketing), employs someone who is less competent in that area. The CEO may do this intentionally to avoid feeling overshadowed, leading to poor performance and outcomes, as exemplified by Michael Eisner’s experience when he took over ABC at Disney and faced challenges, despite his capabilities.

Context Used:

Context 1:
Score: 0.6582
Text: ordingly.
Seventh, when hiring the executive to run your former specialty, be
careful you don’t hire someone weak on purpose.
This sounds silly, but you wouldn’t believe how oaen it happens.
The CEO who used to be a product manager who has a weak
product management executive. The CEO who used to be i

<div>
<h1>🏗️ Activity #1:</h1>

Enhance your RAG application in some way! 


- Refactored and abstracted DocumentManager to handle the documen splitter and vector db creation
- Allow it to work with PDF files
    - Single pdf
    - Folder with mixed pdf and txt files
- Implement a new distance metric
    - Euclidian Distance
    - Minkowski Distance

- Future - Add metadata support to the vector database
- Future - Change splitting algorithm
</div>


> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

The reading of the document, the chunking, and creation of the vector database
have been moved to the DocumentManager class to improve repeatability

In [71]:

import importlib
from gower import documentmanager
importlib.reload(documentmanager)

document_manager = documentmanager.DocumentManager()

vector_db_saga = document_manager.prepare_document("data/eiriks_saga_rauda.en.txt")
print(vector_db_saga.vectors)
retrieval_augmented_qa_pipeline_saga = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db_saga,
    llm=chat_openai
)


Docuemnt processed - ready for querying
defaultdict(<built-in function array>, {"\nThe Saga of Erik the Red\n1880 translation into English by J. Sephton from the original Icelandic 'Eiríks saga rauða'.\n    \n\n\nChapter 1\n\nOlaf, who was called Olaf the White, was styled a warrior king. He was the son of King Ingjald, the son of Helgi, the son of Olaf, the son of Gudred, the son of Halfdan Whiteleg, king of the Uplands (in Norway).\n\nHe led a harrying expedition of sea-rovers into the west, and conquered Dublin, in Ireland, and Dublinshire, over which he made himself king. He married Aud the Deep-minded, daughter of Ketil Flatnose, son of Bjorn the Ungartered, a noble man from Norway. Their son was named Thorstein the Red.\n\nOlaf fell in battle in Ireland, and then Aud and Thorstein went into the Sudreyjar (the Hebrides). There Thorstein married Thorid, daughter of Eyvind the Easterling, sister of Helgi the Lean; and they had many children.\n\n Thorstein became a warrior king, and 

In [79]:
result = retrieval_augmented_qa_pipeline_saga.run_pipeline("Who is Olaf?")
response = result["response"]
context_list = result["context"]
print("Response:")
print(response)
print("")
print("---")
print("")
# Print each context with its similarity score
for i, (context, score) in enumerate(context_list, 1):
    print(f"\nContext {i}:")
    print(f"Score: {score:.4f}")
    print(f"Text: {context.strip()[:100]}")

Response:
Olaf, referred to as Olaf the White, was a warrior king and the son of King Ingjald. He is noted for leading a military expedition into the west, conquering Dublin in Ireland, and becoming its king. He married Aud the Deep-minded and had a son named Thorstein the Red. Olaf fell in battle in Ireland.

---


Context 1:
Score: 0.4872
Text: Olaf Tryggvason.

Now, when Leif sailed from Greenland during the summer, he and his men were driven

Context 2:
Score: 0.4402
Text: The Saga of Erik the Red
1880 translation into English by J. Sephton from the original Icelandic 'Ei

Context 3:
Score: 0.3549
Text: e others. Aud replied, "That it was of no moment to him, for," she said, "he would be esteemed in wh

Context 4:
Score: 0.3542
Text: of peerless nobility in all her conduct.

 There was a man named Orm, who dwelt at Arnarstapi (eagle


Add support for pdfs DONE
- Try with .pdf file and see how it fails - make sure it fails gracefully DONE
- Identify code that handles text files - text_utilities.py load() DONE
- Add code to identify file is .pdf DONE
- Add code to convert .pdf into text and push into documents DONE
- Test for a .pdf document DONE

Clean up the code to prepare for automating and measuring different performances

In [80]:
import importlib
from gower import documentmanager
from aimakerspace import text_utils, vectordatabase
importlib.reload(documentmanager)
importlib.reload(text_utils)
importlib.reload(vectordatabase)

def create_pipeline(document_path):
    document_manager = documentmanager.DocumentManager()

    vector_db = document_manager.prepare_document(document_path)
    retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
        vector_db_retriever=vector_db,
        llm=chat_openai
    )
    return retrieval_augmented_qa_pipeline
    
def ask_question(pipeline: RetrievalAugmentedQAPipeline, question: str):
    response = pipeline.run_pipeline(question)
    formatted_response = response["response"].replace(". ", ".\n")
    print("")
    print(question)
    print(formatted_response)
    print("---")

Create the particular pipeline for a document
Then can run multiple queries on the pipeline

In [82]:
import importlib
from gower import documentmanager
from aimakerspace import text_utils
importlib.reload(documentmanager)
importlib.reload(text_utils)

document_manager = documentmanager.DocumentManager()

ooux_pipeline = create_pipeline("data/OOUX_launch_guide.pdf")
saga_pipeline = create_pipeline("data/eiriks_saga_rauda.en.txt")


Docuemnt processed - ready for querying
Docuemnt processed - ready for querying


Ask questions on the pipeline

In [83]:
ask_question(ooux_pipeline, "What do you know about OOUX?")
ask_question(ooux_pipeline, "What do you know about olaf?")
ask_question(saga_pipeline, "What do you know about olaf?")


What do you know about OOUX?
OOUX stands for Object-Oriented User Experience.
It is a movement toward more strategic and collaborative UX practices that employs fun and game-like techniques.
OOUX aims to simplify complex design projects through frameworks like the ORCA Process, which helps in understanding and organizing system architecture.
The community offers resources such as monthly meetups, online courses, and a comprehensive Masterclass that includes video content and templates.
The community also runs a certification program to help UX professionals further their knowledge and skills in OOUX.
---

What do you know about olaf?
I don't know.
---

What do you know about olaf?
Olaf, referred to as Olaf the White, was a warrior king and the son of King Ingjald.
He is notable for leading a harrying expedition of sea-rovers into the west, conquering Dublin in Ireland and Dublinshire, and claiming kingship over that territory.
Olaf married Aud the Deep-minded, who was the daughter of 

Implement a new distance metric - DONE
- Identify code where distance metric is handled
    - Currently using cosine_similarity() in vectordatabase.py, called using the search function
    - Is passed in through the search_by_text() function
    - Can pass in a different search mechanism to search_by_text() function
- Determine new code to calculate distance metric
    - look at Euclidean Distance and Minkowski Distance
    - identify default parameters if any
- Implement new code
    - created code for Euclidean Distance and Minkowski Distance    
- Write driver to use both code to determine effect
    - Done

In [84]:
import numpy as np
def euclidean_distance(vector_a: np.array, vector_b: np.array) -> float:
    """Computes the Euclidean distance between two vectors."""
    distance = np.linalg.norm(vector_a - vector_b)
    return distance

def minkowski_distance(vector_a: np.array, vector_b: np.array, p: float = 3) -> float:
    """Computes the Minkowski distance between two vectors for a given order p.
        Default p=3 - a distance between Manhattan and Euclidean
        Note    1=Manhattan distance
                2=Euclidean distance
    """
    distance = np.sum(np.abs(vector_a - vector_b) ** p) ** (1 / p)
    return distance

def print_array(in_array):
    for item in in_array:
        print(item)

print("Default - Cosine Similarity")
print_array(vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", 
                         k=5))
print(" ")
print("Euclidean Distance")
print_array(vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", 
                         k=5, distance_measure=euclidean_distance))
print(" ")
print("Minkowski Distance")
print_array(vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", 
                         k=5, distance_measure=minkowski_distance))

Default - Cosine Similarity
('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\

Future Enhancement:

- Add metadata support to the vector database
- Improve chunking mechanism
- Add performance statistics